## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras import regularizers

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data() # x是圖片,y是答案label

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
#Cifar-10 的所有圖片被分為 10 個類別 (以 0~9 數字作為 Label 之編碼) :
#0 : airplain (飛機)
#1 : automobile (汽車)
#2 : bird (鳥)
#3 : cat (貓)
#4 : deer (鹿)
#5 : dog (狗)
#6 : frog (青蛙)
#7 : horse (馬)
#8 : ship (船)
#9 : truck (卡車)
(x_train, y_train), (x_test, y_test) = cifar10.load_data() # x是圖片,y是答案label
import matplotlib.pyplot as plt

def plot_image(image):
    fig=plt.gcf()
    fig.set_size_inches(2, 2)                          
    plt.imshow(image, cmap='binary')
    plt.show()

plot_image(x_train[0])
print(y_train[0]) 

<Figure size 200x200 with 1 Axes>

[6]


In [4]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Users\User\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [5]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,(3,3),input_shape=(32,32,3),activation='relu'))
classifier.add(BatchNormalization())##BatchNormalization
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32,(3,3),activation='relu'))
classifier.add(BatchNormalization())##BatchNormalization
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(units=100,activation='relu',kernel_regularizer=regularizers.l2(l=0.001))) ##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization
classifier.add(Dropout(rate=0.5)) ##Dropout

classifier.add(Dense(units=100,activation='relu',kernel_regularizer=regularizers.l2(0.001)))##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization

classifier.add(Dropout(rate=0.3))##Dropout

classifier.add(Dense(units=10,activation='softmax'))
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
from keras.preprocessing.image import ImageDataGenerator ##Augmentation
img_gen = ImageDataGenerator( featurewise_center=True,featurewise_std_normalization=True,rotation_range=10,width_shift_range=0.1,
                                            height_shift_range=0.1,shear_range=0.1,zoom_range=0.1,horizontal_flip=True,vertical_flip=False,dtype=np.float32)
img_gen.fit(x_train)
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='test_loss', patience=8, verbose=1) ##earlystop














Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [9]:
from __future__ import absolute_import, division, print_function, unicode_literals
import sys
import tensorflow as tf
print(sys.version)#3.7.3
print(tf.__version__)
print(tf.test.is_gpu_available())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
2.1.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
Num GPUs Available:  1


RuntimeError: Device placement logging must be set at program startup

In [6]:
##開始訓練
classifier.fit_generator(img_gen.flow(x_train, y_train, batch_size=100) ,steps_per_epoch=500,
                               epochs=100, validation_data = (x_test, y_test),callbacks = [earlystop])
#classifier.fit_generator(img_gen.flow(x_train, y_train, batch_size=10) ,steps_per_epoch=100,
                               #epochs=10, validation_data = (x_test, y_test))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
500/500 [==============================] - 20s 40ms/step - loss: 2.1206 - acc: 0.3515 - val_loss: 13.1222 - val_acc: 0.1835
Epoch 2/100
  7/500 [..............................] - ETA: 9s - loss: 1.7853 - acc: 0.4357 

C:\Users\User\Anaconda3\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `test_loss` which is not available. Available metrics are: val_loss,val_acc,loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


500/500 [==============================] - 18s 36ms/step - loss: 1.6634 - acc: 0.4818 - val_loss: 12.0219 - val_acc: 0.2360
Epoch 3/100
500/500 [==============================] - 18s 36ms/step - loss: 1.4717 - acc: 0.5423 - val_loss: 11.6672 - val_acc: 0.2625
Epoch 4/100
500/500 [==============================] - 18s 36ms/step - loss: 1.3585 - acc: 0.5789 - val_loss: 11.0194 - val_acc: 0.2956
Epoch 5/100
500/500 [==============================] - 18s 36ms/step - loss: 1.2890 - acc: 0.5958 - val_loss: 11.9785 - val_acc: 0.2407
Epoch 6/100
500/500 [==============================] - 18s 36ms/step - loss: 1.2368 - acc: 0.6131 - val_loss: 12.4327 - val_acc: 0.2177
Epoch 7/100
500/500 [==============================] - 18s 36ms/step - loss: 1.2115 - acc: 0.6202 - val_loss: 12.3989 - val_acc: 0.2238
Epoch 8/100
500/500 [==============================] - 18s 36ms/step - loss: 1.1895 - acc: 0.6279 - val_loss: 12.9185 - val_acc: 0.1929
Epoch 9/100
500/500 [==============================] - 18s 3

Epoch 62/100
500/500 [==============================] - 18s 36ms/step - loss: 0.9842 - acc: 0.7038 - val_loss: 9.8701 - val_acc: 0.3405
Epoch 63/100
500/500 [==============================] - 18s 36ms/step - loss: 0.9859 - acc: 0.7037 - val_loss: 10.3190 - val_acc: 0.3216
Epoch 64/100
500/500 [==============================] - 18s 36ms/step - loss: 0.9891 - acc: 0.7029 - val_loss: 11.2561 - val_acc: 0.2714
Epoch 65/100
500/500 [==============================] - 18s 36ms/step - loss: 0.9852 - acc: 0.7040 - val_loss: 10.4276 - val_acc: 0.3189
Epoch 66/100
500/500 [==============================] - 18s 36ms/step - loss: 0.9869 - acc: 0.7036 - val_loss: 10.7107 - val_acc: 0.3060
Epoch 67/100
500/500 [==============================] - 18s 36ms/step - loss: 0.9848 - acc: 0.7027 - val_loss: 10.8304 - val_acc: 0.2961
Epoch 68/100
500/500 [==============================] - 18s 36ms/step - loss: 0.9841 - acc: 0.7033 - val_loss: 10.3305 - val_acc: 0.3223
Epoch 69/100
500/500 [====================

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [7]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[0.01225243, 0.01201329, 0.39436224, 0.14676644, 0.08438788,
        0.04454466, 0.10317219, 0.00315163, 0.19650954, 0.00283965]],
      dtype=float32)